In [1]:
import networkx as nx
import freeman as fm

In [2]:
g = fm.load('marvel.gml')

In [3]:
g.label_nodes('name')
g.set_all_nodes(size=15, labpos='hover')
g.set_all_edges(color=(0, 0, 0, 0.5))

In [4]:
bc = nx.betweenness_centrality(g)
# cc = nx.closeness_centrality(g)
# ec = nx.eigenvector_centrality(g, max_iter=1000)

In [5]:
import pandas as pd
data = pd.DataFrame({
    'id' : [i for i in g.nodes],
    'Name': [g.nodes[i]['name'] for i in g.nodes],
    'ConnAmnt': [g.nodes[i]['amount'] for i in g.nodes],
    'Intermediation (r)': [i for i in bc.values()],
#     'closeness': [i for i in cc.values()],
#     'eigenvector': [i for i in ec.values()]
})

In [6]:
char_infos = pd.read_csv('../dataset/charcters_stats.csv', index_col=0)

In [7]:
semi_df = pd.merge(data, char_infos, how='inner', on=['Name']) 

In [8]:
import numpy as np
semi_df = semi_df.replace('nan', np.nan)
semi_df = semi_df[semi_df['ConnAmnt'].notna()]
semi_df['ConnAmnt'] = pd.to_numeric(semi_df['ConnAmnt'])

In [9]:
semi_df = semi_df.drop_duplicates(subset=['Name'], keep='first')

In [10]:
# import seaborn as sns
# sns.histplot(semi_df, x='Strength', common_norm = True, stat='probability', cumulative=True,bins=[i for i in range(100)])

In [11]:
import statsmodels.api as sm

model = sm.OLS(semi_df['Intermediation (r)'], semi_df[['Intelligence', 'Strength', 'Power']])
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:     Intermediation (r)   R-squared (uncentered):                   0.447
Model:                            OLS   Adj. R-squared (uncentered):              0.406
Method:                 Least Squares   F-statistic:                              11.04
Date:                Sat, 14 Nov 2020   Prob (F-statistic):                    1.91e-05
Time:                        16:46:52   Log-Likelihood:                          101.22
No. Observations:                  44   AIC:                                     -196.4
Df Residuals:                      41   BIC:                                     -191.1
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intelligence     0.0006      0.000      3.331      0.002       0.000       0.001
Strength      2.268e-05      0.000      0.155      0.878      -0.000       0.000
Power           -0.0003      0.000     -1.322      0.193      -0.001       0.000
==============================================================================
Omnibus:                        4.700   Durbin-Watson:                   1.260
Prob(Omnibus):                  0.095   Jarque-Bera (JB):                4.445
Skew:                           0.769   Prob(JB):                        0.108
Kurtosis:                       2.754   Cond. No.                         7.24
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [12]:
g.draw()